# SS6 ssbpのjson、message_pack変換

ss6sdk内のコンバータを改造してssbp相当のデータを持ったjsonを吐き出させる。また、そのjsonをmessage packに変換する。

# 前提

作業対象のファイル一式は、ホスト上の一式をjupyterコンテナに共有させて用意する。

```
git clone --depth 1 https://github.com/SpriteStudio/SpriteStudio6-SDK.git
```

相当のを、コンテナ起動時にマウントする。

```
docker run -d -v c:/Users/mnaga/Documents/SpriteStudio6-SDK:/notebooks/ss6sdk \
    -p 8888:8888 --name ss6sdknb niicloudoperation/notebook
```


# 準備

ビルド環境整備と、message packへのコンバータの用意と、ファイル一式のUTF8BOM+CRLF化

ビルド環境整備。gccとcmakeとnkfを入れる

In [ ]:
!sudo apt-get update && sudo apt-get upgrade -y && \
sudo apt-get install -y build-essential cmake nkf && \
sudo apt-get clean all && sudo apt-get autoremove

message packへのコンバータの用意

In [ ]:
!git clone --depth 1 https://github.com/ludocode/msgpack-tools.git
!cd msgpack-tools && ./configure && make && sudo make install
!json2msgpack -v

文字コードと改行記号を確認し、UTF8BOM+CRLFに変換する。

In [ ]:
!find . -type f \( -name \*.h -or -name \*.c -or -name \*.cpp -or -name \*.cc -or -name \*.txt \) \
| xargs -I{} sh -c "echo -n {} ' ' && nkf --guess {}"

In [ ]:
!find . -type f \( -name \*.h -or -name \*.c -or -name \*.cpp -or -name \*.cc -or -name \*.txt \) \
| xargs -I{} sh -c "nkf -w8Lw --overwrite {}"

In [ ]:
!find . -type f \( -name \*.h -or -name \*.c -or -name \*.cpp -or -name \*.cc -or -name \*.txt \) \
| xargs -I{} sh -c "echo -n {} ' ' && nkf --guess {}"

# ssbpコンバータのビルド

In [ ]:
%%bash
cd Build
rm -f build
mkdir build
cd build
cmake ..
make Ss6Converter
./Ss6Converter

# コンバート

In [ ]:
%%bash

echo "-- allAttributeV6 --"
cd Build/TestData/allAttributeV6
../../build/Ss6Converter allAttributeV6.sspj
../../build/Ss6Converter -f json allAttributeV6.sspj
python -m json.tool < allAttributeV6.ssbp.json > allAttributeV6.ssbp.formated.json
json2msgpack -bli allAttributeV6.ssbp.json -o allAttributeV6.ssbp.mp
ls -la *.ssbp*

echo "-- allPartsV6 --"
cd ../allPartsV6
../../build/Ss6Converter allPartsV6.sspj
../../build/Ss6Converter -f json allPartsV6.sspj
python -m json.tool < allPartsV6.ssbp.json > allPartsV6.ssbp.formated.json
json2msgpack -bli allPartsV6.ssbp.json -o allPartsV6.ssbp.mp
ls -la *.ssbp*

echo "-- effect_allcheck --"
cd ../effect_allcheck
../../build/Ss6Converter effect_allcheck.sspj
../../build/Ss6Converter -f json effect_allcheck.sspj
python -m json.tool < effect_allcheck.ssbp.json > effect_allcheck.ssbp.formated.json
json2msgpack -bli effect_allcheck.ssbp.json -o effect_allcheck.ssbp.mp
ls -la *.ssbp*
